In [31]:
import pandas as pd
from collections import Counter
import torch
from sklearn.model_selection import train_test_split
from networkx.readwrite.json_graph import adjacency
import random, math, time, os
import torch.nn.functional as F
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader



In [32]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [33]:
dataset_pd = pd.read_csv('D:\\VideoRecSystem\\MicroLens\\DataSet\\MicroLens-50k_pairs.csv')
# dataset_pd = pd.read_csv('MicroLens-50k_pairs.csv')

In [34]:
dataset_pd.head(10)

,user,item,timestamp
0,36121,9580,1583378629552
1,26572,9580,1583436719018
2,37550,9580,1584412681021
3,14601,9580,1584848802432
4,15061,9580,1585388171106
5,6364,9580,1585390736041
6,3542,9580,1585404918503
7,21038,9580,1590144594477
8,12538,14631,1634867362929
9,47592,14631,1634872254913


In [35]:
dataset_pd.count

<bound method DataFrame.count of          user   item      timestamp
0       36121   9580  1583378629552
1       26572   9580  1583436719018
2       37550   9580  1584412681021
3       14601   9580  1584848802432
4       15061   9580  1585388171106
...       ...    ...            ...
359703  48702   1363  1662984066647
359704  27203   7291  1662984082974
359705  29261  19649  1662984103874
359706  28341  19188  1662984123833
359707  38967   7254  1662984132429

[359708 rows x 3 columns]>

In [36]:
user_counts = dataset_pd['user'].value_counts()
item_counts = dataset_pd['item'].value_counts()
# valid_users = user_counts[user_counts > 3].index
# valid_items = item_counts[item_counts > 3].index
# filtered_df = dataset_pd[dataset_pd['user'].isin(valid_users) & dataset_pd['item'].isin(valid_items)]
# filtered_df.count

In [37]:
# order by user,timestamp 
filtered_df = dataset_pd.sort_values(by=["user", "timestamp"])


In [38]:
def split(df, user_col='user', item_col='item', time_col='timestamp'):

    df = df.sort_values(by=[user_col, time_col])  # 按用户时间排序

    # 获取每个用户的最后一条记录作为 test
    test_df = df.groupby(user_col).tail(1)
    train_df = df.drop(index=test_df.index)

    # 过滤 test 中那些 user/item 不在 train 中的
    train_users = set(train_df[user_col])
    train_items = set(train_df[item_col])

    test_df = test_df[
        test_df[user_col].isin(train_users) &
        test_df[item_col].isin(train_items)
    ]

    return train_df.reset_index(drop=True), test_df.reset_index(drop=True)

In [39]:

train_df, test_df = split(filtered_df,user_col='user', item_col='item', time_col='timestamp')

print(f"Train size: {len(train_df)}")
print(f"Test size: {len(test_df)}")


Train size: 309708
Test size: 49424


In [40]:
# maintain a map from new id to old id, new id for constructing matrix
user2id = {u: i for i, u in enumerate(filtered_df['user'].unique())}
item2id = {i: j for j, i in enumerate(filtered_df['item'].unique())}

# apply to train_df and test_df
train_df['user_id'] = train_df['user'].map(user2id)
train_df['item_id'] = train_df['item'].map(item2id)
test_df['user_id'] = test_df['user'].map(user2id)
test_df['item_id'] = test_df['item'].map(item2id)

num_users = len(user2id)
num_items = len(item2id)


In [41]:
import lmdb

def load_lmdb_embeddings(lmdb_path, num_items, emb_dim=128):
    """读取 LMDB 中的 128 维 cover 向量，key 从 1 开始计数"""
    env = lmdb.open(lmdb_path, readonly=True, lock=False,subdir=False)
    cover_embs = np.zeros((num_items + 1, emb_dim), dtype=np.float32)  # index 0 保留给 PAD
    with env.begin() as txn:
        for idx in range(1, num_items + 1):
            val = txn.get(str(idx).encode('ascii'))
            if val is not None:
                cover_embs[idx] = np.frombuffer(val, dtype=np.float32)
    env.close()
    return torch.tensor(cover_embs)

COVER_EMB_PATH = r"D:/VideoRecSystem/MicroLens/cover_emb128.lmdb"
print(f"Loading cover embeddings from {COVER_EMB_PATH} ...")
COVER_EMBS = load_lmdb_embeddings(COVER_EMB_PATH, num_items=num_items, emb_dim=128)
print("COVER_EMBS shape:", COVER_EMBS.shape)  # (num_items+1, 128)

Loading cover embeddings from D:/VideoRecSystem/MicroLens/cover_emb128.lmdb ...
COVER_EMBS shape: torch.Size([19221, 128])


In [42]:
# ---------- 超参数 ----------
MAX_SEQ_LEN   = 20          # 序列长度
EMBED_DIM     = 64          # item / user embedding 维度
N_HEADS       = 2           # Multi-Head Attention 头数
N_LAYERS      = 2           # Transformer block 层数
DROPOUT       = 0.2
NEG_SAMPLE    = 5
BATCH_SIZE    = 512
EPOCHS        = 10
LR            = 1e-3
SEED          = 42
# ----------------------------

torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

# ---------- 常量（来自你已有变量） ----------
PAD_IDX = num_items          # 专用 padding id
N_ITEMS = num_items + 1      # Embedding 行数（含 PAD）
ALL_ITEM_IDS = np.arange(num_items, dtype=np.int64)
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# -------------------------------------------

# ======== Dataset (同前) ======== #
# ======== Dataset：与前一致 ======== #
class SASRecBPRDataset(Dataset):
    def __init__(self, df, max_len=MAX_SEQ_LEN, pad_idx=PAD_IDX, n_neg=NEG_SAMPLE):
        self.max_len, self.pad_idx, self.n_neg = max_len, pad_idx, n_neg
        self.inputs, self.targets = [], []

        for _, hist in df.groupby('user_id'):
            seq = hist['item_id'].tolist()
            for i in range(1, len(seq)):
                s = seq[max(0, i-max_len): i]
                s = [pad_idx]*(max_len-len(s)) + s
                self.inputs.append(s)
                self.targets.append(seq[i])

        self.inputs  = np.asarray(self.inputs,  dtype=np.int64)
        self.targets = np.asarray(self.targets, dtype=np.int64)

    def __len__(self): return len(self.targets)

    def _neg(self, pos):
        negs = np.random.choice(ALL_ITEM_IDS, size=self.n_neg, replace=False)
        while (negs == pos).any():
            dup = negs == pos
            negs[dup] = np.random.choice(ALL_ITEM_IDS, size=dup.sum(), replace=False)
        return negs

    def __getitem__(self, idx):
        hist = torch.tensor(self.inputs[idx], dtype=torch.long)
        pos  = torch.tensor(self.targets[idx], dtype=torch.long)
        negs = torch.tensor(self._neg(self.targets[idx]), dtype=torch.long)
        return hist, pos, negs



In [43]:
# ======== SASRec 模型 ======== #
class SASRec(nn.Module):
    def __init__(self,
                 cover_embs=None,
                 n_items=N_ITEMS,
                 dim=EMBED_DIM,
                 n_heads=N_HEADS,
                 n_layers=N_LAYERS,
                 max_len=MAX_SEQ_LEN,
                 pad_idx=PAD_IDX,
                 dropout=DROPOUT):

        super().__init__()

        # ① ID embedding（可训练）
        self.id_emb   = nn.Embedding(n_items, dim, padding_idx=pad_idx)
        # ② cover embedding（冻结）
        self.cover_emb = nn.Embedding.from_pretrained(cover_embs, freeze=True)

        # ③ 将拼接后的 (dim+128) → dim 的 1×1 线性层
        self.in_proj = nn.Linear(dim + 128, dim, bias=False)


        self.pos_emb  = nn.Embedding(max_len, dim)
        self.dropout  = nn.Dropout(dropout)

        enc_layer = nn.TransformerEncoderLayer(
            d_model=dim,
            nhead=n_heads,
            dim_feedforward=dim*4,
            dropout=dropout,
            batch_first=True,
            activation='gelu')
        self.encoder = nn.TransformerEncoder(enc_layer, num_layers=n_layers)

        # 为 causal mask 预生成上三角矩阵
        self.register_buffer(
            "mask", torch.triu(torch.ones(max_len, max_len), diagonal=1).bool())

    def forward(self, seq):                       # seq : (B,T)
        B, T = seq.size()
        id_emb   = self.id_emb(seq)              # (B, T, D)
        cover_vec= self.cover_emb(seq)           # (B, T, 128)
        x = torch.cat([id_emb, cover_vec], -1)   # (B, T, D+128)
        x = self.in_proj(x)                      # (B, T, D) —— 轻量压缩


        pos_ids = torch.arange(T, device=seq.device).unsqueeze(0).expand(B, -1)
        x = x + self.pos_emb(pos_ids)        # 加位置编码
        x = self.dropout(x)

        # causal attention mask
        causal_mask = self.mask[:T, :T]
        x = self.encoder(x, src_key_padding_mask=(seq == PAD_IDX), mask=causal_mask)
        h = x[:, -1, :]                           # 取最后位置向量 (B,D)
        return h

    def score(self, h, items):
        # item embedding 拼接后投影
        item_id_vec = self.id_emb(items)         # (B, D)
        item_cover_vec = self.cover_emb(items)   # (B, 128)
        item_vec = torch.cat([item_id_vec, item_cover_vec], dim=-1)
        item_vec = self.in_proj(item_vec)        # (B, D)

        # === 加入 L2 规范化 ===
        h = F.normalize(h, p=2, dim=-1)               # (B, D)
        item_vec = F.normalize(item_vec, p=2, dim=-1) # (B, D)

        return (h.unsqueeze(-2) * item_vec).sum(-1)   # cosine similarity


In [44]:
# ======== BPR 损失 ======== #
def bpr_loss(pos_s, neg_s):
    return -torch.log(torch.sigmoid(pos_s.unsqueeze(-1) - neg_s) + 1e-8).mean()



In [45]:
# ======== 训练函数 ======== #
def train_sasrec_bpr(train_df, test_df=None):
    ds = SASRecBPRDataset(train_df)
    loader = DataLoader(ds, batch_size=BATCH_SIZE, shuffle=True,
                        num_workers=0, pin_memory=True)

    model = SASRec(cover_embs=COVER_EMBS).to(DEVICE)
    optim = torch.optim.Adam(model.parameters(), lr=LR)

    for ep in range(1, EPOCHS+1):
        model.train()
        tot_loss, t0 = 0.0, time.time()
        for step, (hist, pos, neg) in enumerate(loader, 1):
            hist, pos, neg = hist.to(DEVICE), pos.to(DEVICE), neg.to(DEVICE)
            h = model(hist)
            pos_s = model.score(h, pos)
            neg_s = model.score(h, neg)
            loss  = bpr_loss(pos_s, neg_s)

            optim.zero_grad()
            loss.backward()
            optim.step()

            tot_loss += loss.item()*hist.size(0)
            if step % 100 == 0 or step == 1:
                print(f"[Ep {ep}] step {step}/{len(loader)} | loss {loss.item():.4f}", flush=True)

        print(f"Ep {ep} done | avg loss {tot_loss/len(ds):.4f} | {time.time()-t0:.1f}s\n", flush=True)
    return model

In [46]:
# ======== 构建 (user → 历史张量) 缓存 ======== #
def build_hist_cache(df):
    cache = {}
    for uid, items in df.groupby('user_id')['item_id']:
        seq = items.tolist()[-MAX_SEQ_LEN:]
        seq = [PAD_IDX]*(MAX_SEQ_LEN - len(seq)) + seq
        cache[uid] = torch.tensor(seq, dtype=torch.long).unsqueeze(0)
    return cache

In [47]:
num_users

50000

In [48]:

def evaluate_ranking(
        test_df,              # DataFrame, 必含 user_id / item_id
        train_df,             # DataFrame, 用来构建用户→已交互物品集合
        score_fn,             # callable(users_tensor, items_tensor) → np.array
        num_items,            # 物品总数
        k=10,                 # Hit@K / NDCG@K
        num_neg=10000,          # 每个正样本采多少负样本
        user_col='user_id',
        item_col='item_id',
        seed=42
    ):
    """
    不依赖具体模型，只要提供 score_fn 就能评估。
    score_fn: 接收 (user_tensor, item_tensor) 并返回同长度的 Numpy 分数向量。
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

    # 用户历史，用于采负样本 & 过滤
    train_user_dict = (
        train_df.groupby(user_col)[item_col].apply(set).to_dict()
    )

    hits, ndcgs = [], []

    for _, row in test_df.iterrows():
        u = int(row[user_col])
        pos_item = int(row[item_col])

        # ---------- 负采样 ----------
        neg_items = set()
        while len(neg_items) < num_neg:
            neg = random.randint(0, num_items - 1)
            if neg not in train_user_dict.get(u, set()) and neg != pos_item:
                neg_items.add(neg)

        item_candidates = list(neg_items) + [pos_item]

        # ---------- 评分 ----------
        users_t  = torch.LongTensor([u] * len(item_candidates))
        items_t  = torch.LongTensor(item_candidates)
        scores   = score_fn(users_t, items_t)        # ← 只依赖 score_fn
        rank_idx = np.argsort(scores)[::-1]          # 降序
        ranked_items = [item_candidates[i] for i in rank_idx]

        # ---------- 指标 ----------
        if pos_item in ranked_items[:k]:
            hits.append(1)
            rank_pos = ranked_items.index(pos_item)
            ndcgs.append(1 / np.log2(rank_pos + 2))
        else:
            hits.append(0)
            ndcgs.append(0)

    hit_rate = float(np.mean(hits))
    ndcg     = float(np.mean(ndcgs))
    return hit_rate, ndcg

In [49]:
    # ------------------ 训练 ------------------
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model  = train_sasrec_bpr(train_df, test_df)

[Ep 1] step 1/508 | loss 0.7014
[Ep 1] step 100/508 | loss 0.6991
[Ep 1] step 200/508 | loss 0.6710
[Ep 1] step 300/508 | loss 0.6583
[Ep 1] step 400/508 | loss 0.6399
[Ep 1] step 500/508 | loss 0.5916
Ep 1 done | avg loss 0.6560 | 190.8s

[Ep 2] step 1/508 | loss 0.5756
[Ep 2] step 100/508 | loss 0.5705
[Ep 2] step 200/508 | loss 0.5631
[Ep 2] step 300/508 | loss 0.5554
[Ep 2] step 400/508 | loss 0.5440
[Ep 2] step 500/508 | loss 0.5611
Ep 2 done | avg loss 0.5510 | 190.9s

[Ep 3] step 1/508 | loss 0.5235
[Ep 3] step 100/508 | loss 0.5029
[Ep 3] step 200/508 | loss 0.5107
[Ep 3] step 300/508 | loss 0.5192
[Ep 3] step 400/508 | loss 0.5047
[Ep 3] step 500/508 | loss 0.5132
Ep 3 done | avg loss 0.5181 | 190.0s

[Ep 4] step 1/508 | loss 0.5191
[Ep 4] step 100/508 | loss 0.5208
[Ep 4] step 200/508 | loss 0.5210
[Ep 4] step 300/508 | loss 0.5091
[Ep 4] step 400/508 | loss 0.5022
[Ep 4] step 500/508 | loss 0.4900
Ep 4 done | avg loss 0.5084 | 191.6s

[Ep 5] step 1/508 | loss 0.4888
[Ep 5] s

In [50]:
def make_popularity_score_fn(train_df, item_col='item_id'):
    item_cnt = Counter(train_df[item_col])
    default_score = min(item_cnt.values()) - 1  # 给没出现过的物品一个更低分
    def _score_fn(users_t, items_t):
        return np.array([item_cnt.get(int(i), default_score) for i in items_t])
    return _score_fn

In [51]:
def random_score_fn(users_t, items_t):
    # 随机给每个 items_t 一个分数；users_t 不使用，但必须接收
    return np.random.rand(len(items_t))

In [61]:
def make_score_fn(model, hist_cache):
    """
    内存优化版：将 item embedding 缓存在 CPU，仅在使用时搬到 GPU。
    """
    model.eval()
    device = next(model.parameters()).device

    with torch.no_grad():
        id_mat    = model.id_emb.weight.data.cpu()       # (n_items+1, D)
        cover_mat = model.cover_emb.weight.data.cpu()    # (n_items+1, 128)

        # ——— 将 in_proj 暂时移动到 CPU 计算后再还原 ——— #
        in_proj_cpu = model.in_proj.cpu()  # 临时版本

        fused_mat = in_proj_cpu(
            torch.cat([id_mat, cover_mat], dim=1)
        ).cpu()  # 强制保持在 CPU

        model.in_proj.to(device)  # 还原原模型的 in_proj 到 CUDA

        fused_mat = F.normalize(fused_mat, p=2, dim=-1)

        print(f"[INFO] fused_mat.shape = {fused_mat.shape}, "
              f"CPU memory = {fused_mat.element_size() * fused_mat.nelement() / 1e6:.2f} MB")
    @torch.no_grad()
    def score_fn(user_t, item_t):
        """
        user_t : (m,) 全为同一个用户 ID
        item_t : (m,) 待评分 item ids
        """
        uid = int(user_t[0])
        hist_seq = hist_cache[uid].to(device)  # (1, T)

        h = model(hist_seq)                    # (1, D)
        h = F.normalize(h, p=2, dim=-1)        # (1, D)

        # ——— 仅搬运用到的 item 表到 GPU ——— #
        item_vec = fused_mat[item_t.cpu()]     # (m, D) on CPU
        item_vec = item_vec.to(device)         # 搬到 CUDA

        scores = (h * item_vec).sum(-1)        # (m,)
        return scores.cpu().numpy()

    return score_fn


In [ ]:
def make_score_fn_GPU(model, hist_cache):
    """
    正确设备统一版本，完全在 CUDA 上处理，显存可控。
    """
    model.eval()
    device = next(model.parameters()).device

    # —— 全在 CUDA 上构建 item 表征 —— #
    with torch.no_grad():
        id_mat    = model.id_emb.weight.to(device)         # (n_items+1, D)
        cover_mat = model.cover_emb.weight.to(device)      # (n_items+1, 128)

        fused_mat = model.in_proj(
            torch.cat([id_mat, cover_mat], dim=1)          # → (n_items+1, D+128)
        )  # → (n_items+1, D)
        fused_mat = F.normalize(fused_mat, p=2, dim=-1)

        print(f"[INFO] fused_mat on CUDA: shape = {fused_mat.shape}, "
              f"GPU memory = {fused_mat.element_size() * fused_mat.nelement() / 1e6:.2f} MB")

    @torch.no_grad()
    def score_fn(user_t, item_t):
        uid = int(user_t[0])
        hist_seq = hist_cache[uid].to(device)     # (1, T)

        h = model(hist_seq)                       # (1, D)
        h = F.normalize(h, p=2, dim=-1)

        item_vec = fused_mat[item_t.to(device)]   # (m, D)
        scores = (h * item_vec).sum(-1)
        return scores.cpu().numpy()

    return score_fn

In [62]:

# ----------------SASRec （或其他模型）------------
score_fn_SASRec = make_score_fn(model,hist_cache=build_hist_cache(train_df))
hit_SASRec, ndcg_SASRec = evaluate_ranking(
    test_df, train_df, score_fn_SASRec,
    num_items=num_items, k=10
)
print(f"SASRec   Hit@10={hit_SASRec:.4f}  NDCG@10={ndcg_SASRec:.4f}")

# ---------------- baseline：Popular ----------------
pop_score_fn  = make_popularity_score_fn(train_df)
hit_pop, ndcg_pop = evaluate_ranking(
    test_df, train_df, pop_score_fn,
    num_items=num_items, k=10
)
print(f"Popular  Hit@10={hit_pop:.4f}  NDCG@10={ndcg_pop:.4f}")

# ---------------- baseline：Random -----------------
hit_rand, ndcg_rand = evaluate_ranking(
    test_df, train_df, random_score_fn,
    num_items=num_items, k=10
)
print(f"Random   Hit@10={hit_rand:.4f}  NDCG@10={ndcg_rand:.4f}")



[INFO] fused_mat.shape = torch.Size([19221, 64]), CPU memory = 4.92 MB
SASRec   Hit@10=0.9846  NDCG@10=0.9845


KeyboardInterrupt: 